**Processing K's Images**<br>
Resize, cutting into square patches for DeepCell training

In [1]:
import syotil

import numpy as np
from skimage import io
from matplotlib import pyplot as plt
%matplotlib inline
from timeit import default_timer
import os

import tensorflow as tf

from deepcell_toolbox.utils import resize, tile_image, untile_image

In [2]:
# read image file and put data into tensors
import glob
INPUT_PATH="images/training/"
#INPUT_PATH="images/training_512x512/"
FILENAMES = glob.glob(INPUT_PATH+"*_img.png")
print(len(FILENAMES))
print(FILENAMES)

imgs = [io.imread(CURR_IM_NAME)[:,:,0] for CURR_IM_NAME in FILENAMES]
X_train = tf.stack(imgs)
X_train = np.expand_dims(X_train, axis=-1)
X_train = np.array(X_train, dtype=np.uint8) # tf.stack changes the dtype 
print(X_train.shape)

masks = [io.imread(CURR_IM_NAME.replace("img","masks")) for CURR_IM_NAME in FILENAMES]
y_train = tf.stack(masks)
y_train = np.expand_dims(y_train, axis=-1)
y_train = np.array(y_train, dtype=np.uint16)  # tf.stack changes the dtype, change it back here so that when saving to files, there will be no loss
print(y_train.shape)

7
['images/training/M872956_JML_Position9_CD3_train_img.png', 'images/training/M872956_JML_Position8_CD3_train_img.png', 'images/training/M926910_CFL_Position7_CD3_train_img.png', 'images/training/M926910_CFL_Position13_CD3_train_img.png', 'images/training/M872956_JML_Position8_CD8_train_img.png', 'images/training/M872956_JML_Position10_CD3_train_img.png', 'images/training/M872956_JML_Position8_CD4_train_img.png']


2022-11-05 14:04:48.140237: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14657 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:04:00.0, compute capability: 7.0
2022-11-05 14:04:48.141678: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14657 MB memory:  -> device: 1, name: Tesla V100-PCIE-16GB, pci bus id: 0000:06:00.0, compute capability: 7.0
2022-11-05 14:04:48.142875: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 14657 MB memory:  -> device: 2, name: Tesla V100-PCIE-16GB, pci bus id: 0000:08:00.0, compute capability: 7.0


(7, 1040, 1159, 1)
(7, 1040, 1159, 1)


**Cut each image into 25 512x512 patches**

In [ ]:
# define cut lines
x_starts=[0,120,120*2,120*3,X_train.shape[1]-512]
x_ends=[x+512 for x in x_starts]
print(x_starts)
print(x_ends)

y_starts=[0,150,150*2,150*3,X_train.shape[2]-512]
y_ends=[x+512 for x in y_starts]
print(y_starts)
print(y_ends)

In [ ]:
# save files
for i in range(len(x_starts)):
    for j in range(len(y_starts)):
        sub_X = X_train[:, x_starts[i]:x_ends[i], y_starts[j]:y_ends[j], :]
        sub_y = y_train[:, x_starts[i]:x_ends[i], y_starts[j]:y_ends[j], :]
        for idx, CURR_IM_NAME in enumerate(FILENAMES):
            fname=CURR_IM_NAME.replace("training","training_512x512_25").replace("train_img","train{}_img".format(j+i*len(y_starts)))
            io.imsave(fname, sub_X[idx,...], check_contrast=False) # use ubyte to suppress warnings
            fname=CURR_IM_NAME.replace("training","training_512x512_25").replace("train_img","train{}_masks".format(j+i*len(y_starts)))
            io.imsave(fname, sub_y[idx,...], check_contrast=False)


**Resize image with mpp_select 1.3**

In [3]:
# resize image
mpp_select=1.3 # based on DeepCell_tn_nuclear.ipynb, the best value is 1.2, but 1.3 makes scale factor an integer
shape = X_train.shape
scale_factor = mpp_select / 0.65
print(scale_factor)
new_shape = (int(shape[1] * scale_factor),
             int(shape[2] * scale_factor))

X_train_resized = resize(X_train, new_shape, data_format='channels_last')
y_train_resized = resize(y_train, new_shape, data_format='channels_last', labeled_image=True)
print(X_train_resized.shape)
print(y_train_resized.shape)

# compare before and after resizing
print(np.histogram(y_train_resized[0,...], bins=np.append(np.unique(y_train_resized[0,...]), np.inf)))
np.histogram(y_train[0,...], bins=np.append(np.unique(y_train[0,...]), np.inf))

2.0
(7, 2080, 2318, 1)
(7, 2080, 2318, 1)
(array([3640464,     660,     984, ...,    1396,     888,     524]), array([0.000e+00, 1.000e+00, 2.000e+00, ..., 1.985e+03, 1.986e+03,
             inf]))


(array([910116,    165,    246, ...,    349,    222,    131]),
 array([0.000e+00, 1.000e+00, 2.000e+00, ..., 1.985e+03, 1.986e+03,
              inf]))

In [ ]:
# write resized images and masks. saved to images/training_resized
for i in range(7):
    fname=FILENAMES[i].replace("_img","_resized_img").replace("training","training_resized")
    io.imsave(fname, X_train_resized[i,:,:,0], check_contrast=False)

for i in range(7):
    fname=FILENAMES[i].replace("_img","_resized_masks").replace("training","training_resized")
    io.imsave(fname, y_train_resized[i,:,:,0], check_contrast=False)

In [4]:
# cut each resized image into 5x5 patches of size 512x512 
print(X_train_resized.shape)
print(y_train_resized.shape)

x_starts=[0,400,400*2,400*3,X_train_resized.shape[1]-512]
x_ends=[x+512 for x in x_starts]
print(x_starts)
print(x_ends)

y_starts=[0,512,512*2,512*3,X_train_resized.shape[2]-512]
y_ends=[x+512 for x in y_starts]
print(y_starts)
print(y_ends)

(7, 2080, 2318, 1)
(7, 2080, 2318, 1)
[0, 400, 800, 1200, 1568]
[512, 912, 1312, 1712, 2080]
[0, 512, 1024, 1536, 1806]
[512, 1024, 1536, 2048, 2318]


In [ ]:
# save files
for i in range(len(x_starts)):
    for j in range(len(y_starts)):
        sub_X = X_train_resized[:, x_starts[i]:x_ends[i], y_starts[j]:y_ends[j], :]
        sub_y = y_train_resized[:, x_starts[i]:x_ends[i], y_starts[j]:y_ends[j], :]
        for idx, CURR_IM_NAME in enumerate(FILENAMES):
            fname=CURR_IM_NAME.replace("training","training_resized_512x512").replace("train_img","train{}_img".format(j+i*len(y_starts)))
            io.imsave(fname, sub_X[idx,...], check_contrast=False) # use ubyte to suppress warnings
            fname=CURR_IM_NAME.replace("training","training_resized_512x512").replace("train_img","train{}_masks".format(j+i*len(y_starts)))
            io.imsave(fname, sub_y[idx,...], check_contrast=False)


In [5]:
# save another version where masks and pixels on the cut lines are removed
for i in range(len(x_starts)):
    for j in range(len(y_starts)):
        sub_X = X_train_resized[:, x_starts[i]:x_ends[i], y_starts[j]:y_ends[j], :]
        sub_y = y_train_resized[:, x_starts[i]:x_ends[i], y_starts[j]:y_ends[j], :]
        for idx, CURR_IM_NAME in enumerate(FILENAMES):
            rm_idx = np.unique(np.concatenate((np.unique(sub_y[idx,:,0,0]), np.unique(sub_y[idx,0,:,0]), np.unique(sub_y[idx,:,-1,0]), np.unique(sub_y[idx,-1,:,0]))))
            for midx in rm_idx[1:]: # remove the first element from rm_idx, which is 0, the background
                coor = np.where(sub_y[idx,:,:,0] == midx)
                sub_X[idx,coor[0], coor[1],0] = 0
                sub_y[idx,coor[0], coor[1],0] = 0

            fname=CURR_IM_NAME.replace("training","training_resized_512x512_a").replace("train_img","train{}_img".format(j+i*len(y_starts)))
            io.imsave(fname, sub_X[idx,...], check_contrast=False) 
            fname=CURR_IM_NAME.replace("training","training_resized_512x512_a").replace("train_img","train{}_masks".format(j+i*len(y_starts)))
            io.imsave(fname, sub_y[idx,...], check_contrast=False)

**Resize image with mpp_select 1.0**

In [ ]:
# resize image with a different mpp
mpp_select=1.0
shape = X_train.shape
scale_factor = mpp_select / 0.65
print(scale_factor)
new_shape = (int(shape[1] * scale_factor),
             int(shape[2] * scale_factor))

X_train_resized = resize(X_train, new_shape, data_format='channels_last')
y_train_resized = resize(y_train, new_shape, data_format='channels_last', labeled_image=True)
print(X_train_resized.shape)
print(y_train_resized.shape)

# compare before and after resizing
print(np.histogram(y_train_resized[0,...], bins=np.append(np.unique(y_train_resized[0,...]), np.inf)))
np.histogram(y_train[0,...], bins=np.append(np.unique(y_train[0,...]), np.inf))

In [ ]:
# write resized images and masks. saved to images/training_resized
for i in range(7):
    fname=FILENAMES[i].replace("training","training_resized1").replace("_img","_resized1_img")
    io.imsave(fname, X_train_resized[i,:,:,0], check_contrast=False)

for i in range(7):
    fname=FILENAMES[i].replace("training","training_resized1").replace("_img","_resized1_masks")
    io.imsave(fname, y_train_resized[i,:,:,0], check_contrast=False)

In [ ]:
# cut each image into 5x5 patches of size 512x512 
print(X_train_resized.shape)
print(y_train_resized.shape)

x_starts=[0,350,350*2,X_train_resized.shape[1]-512]
x_ends=[x+512 for x in x_starts]
print(x_starts)
print(x_ends)

y_starts=[0,400,400*2,X_train_resized.shape[2]-512]
y_ends=[x+512 for x in y_starts]
print(y_starts)
print(y_ends)

In [ ]:
# save file
for i in range(len(x_starts)):
    for j in range(len(y_starts)):
        sub_X = X_train_resized[:, x_starts[i]:x_ends[i], y_starts[j]:y_ends[j], :]
        sub_y = y_train_resized[:, x_starts[i]:x_ends[i], y_starts[j]:y_ends[j], :]
        for idx, CURR_IM_NAME in enumerate(FILENAMES):
            fname=CURR_IM_NAME.replace("training","training_resized1_512x512").replace("train_img","train{}_img".format(j+i*len(y_starts)))
            io.imsave(fname, sub_X[idx,...], check_contrast=False) # use ubyte to suppress warnings
            fname=CURR_IM_NAME.replace("training","training_resized1_512x512").replace("train_img","train{}_masks".format(j+i*len(y_starts)))
            io.imsave(fname, sub_y[idx,...], check_contrast=False)
